In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import tqdm
pd.options.display.max_rows=999

In [0]:
df = pd.read_csv("order_brush_order.csv")
df['event_time'] = pd.to_datetime(df['event_time'])
print(df.info())
print(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   orderid     222750 non-null  int64         
 1   shopid      222750 non-null  int64         
 2   userid      222750 non-null  int64         
 3   event_time  222750 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 6.8 MB
None
(222750, 4)


,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [0]:
df['shopid'].value_counts()

147941492    11703
61556313      6691
54615708      2640
91799978       707
57189823       687
             ...  
158235164        1
147997211        1
166898182        1
194982850        1
189530112        1
Name: shopid, Length: 18770, dtype: int64

In [0]:
A = df[df['shopid']==147941492]

In [0]:
A.sort_values(by=['event_time'],inplace = True)
print(A)
A['event_time'] = A['event_time'] - A['event_time'].iloc[0] # Dung timestamp

               orderid     shopid     userid          event_time
73106   31076433523765  181009364  214208720 2019-12-27 00:20:33
115420  31077003494175  181009364  214208720 2019-12-27 00:30:03
9891    31078405144719  181009364  214208720 2019-12-27 00:53:26
169569  31167197128282  181009364   27847950 2019-12-28 01:33:18
155018  31200690834058  181009364  214515524 2019-12-28 10:51:30
88062   31243796889716  181009364  160784036 2019-12-28 22:49:57
36556   31248330354574  181009364   18028327 2019-12-29 00:05:30
36584   31298070356347  181009364  101832161 2019-12-29 13:54:31
136504  31298092385907  181009364  101832161 2019-12-29 13:54:52
123695  31298150262970  181009364  101832161 2019-12-29 13:55:51
87078   31359530425916  181009364    1646431 2019-12-30 06:58:51
51330   31392356223812  181009364  213441594 2019-12-30 16:05:56
179523  31409023914877  181009364  198488910 2019-12-30 20:43:43
23139   31465898498833  181009364   21653103 2019-12-31 12:31:38
85340   31470607300366  1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
A['second'] = A['event_time'].dt.total_seconds()
A

In [0]:
def solve_interval(interval, fil):
  concen = (interval.shape[0]) / (interval['userid'].nunique())
  if concen >= 2.8:
    fil = fil.append(interval[['orderid','userid']], ignore_index=True, sort=False)
  return fil

In [0]:
def solve(shop):
  if shop.shape[0] < 3:
    return '0'

  shop.sort_values(by=['event_time'],inplace = True)
  shop['event_time'] = shop['event_time'] - shop['event_time'].iloc[0]
  shop['second'] = shop['event_time'].dt.total_seconds()
  r = 0
  fill = pd.DataFrame(None)

  for l in range(shop.shape[0]):
    while (r<shop.shape[0] and shop['second'].iloc[r] - shop['second'].iloc[l] <= 3601):
      r+=1
    fill = solve_interval(shop[l:r],fill)
  
  if fill.shape[0] == 0:
    return '0'
  fill = fill.drop_duplicates(keep = 'last') 
  B = fill['userid'].value_counts()
  max_order = B.iloc[0]
  ans = ""
  for index, value in B.items():
    if value != max_order:
      break
    ans = ans+ str(index)+'&'
  return ans[:-1]

In [0]:
result = []
for gr in tqdm.tqdm(df.groupby('shopid')):
    shop_id = gr[0]
    df_shop = gr[1]
    result.append([shop_id,solve(df_shop)])

 46%|████▌     | 8569/18770 [01:23<01:40, 101.75it/s]

In [0]:
df_res = pd.DataFrame(result,columns=['shopid','userid'])
df_res.to_csv('sub.csv',index=False)

In [0]:
df_res2 = pd.DataFrame(result,columns=['shopid','userid'])

In [0]:
df_res2[df_res2['userid']=='0']

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
...,...,...
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0


In [0]:
df_res[df_res['userid']=='0']

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
...,...,...
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0


In [0]:
for i in range(df_res.shape[0]):
  if df_res['userid'].iloc[i] != df_res2['userid'].iloc[i]:
    print(df_res['userid'].iloc[i], df_res2['userid'].iloc[i])

156202149&214265994 156202149
